In [ ]:


api_key = "sk-cRk2eMr7OMij3FKyD4C41a291c9943018728C3D479Dd1e99"
base_url = "https://chatapi.onechats.top/v1/"


class OpenAIBackend:
    _aclient = None
    _client = None

    @staticmethod
    def config(**kwargs) -> None:
        from openai import OpenAI, AsyncOpenAI
        OpenAIBackend._client = OpenAI(**kwargs)
        OpenAIBackend._aclient = AsyncOpenAI(**kwargs)

    def __init__(self, **kwargs) -> None:
        self.kwargs = kwargs

    async def run(self, messages) -> str:
        response = self._client.chat.completions.create(messages=messages, **self.kwargs)
        return response.choices[0].message.content

    async def arun(self, messages) -> str:
        response = await self._aclient.chat.completions.create(messages=messages, **self.kwargs)
        return response.choices[0].message.content
OpenAIBackend.config(api_key=api_key,base_url=base_url)

In [ ]:
from jinja2 import Template, Environment, FileSystemLoader

class SinglePasser:
    """只有初始化的prompt设定，以及forward生成内容。"""

    def __init__(self, backend: OpenAIBackend, prompt: Template = None, system: str = None, **kwargs):
        self.system = system
        self.prompt = prompt
        self.backend = backend
        self.kwargs = kwargs

    async def forward(self, query: str):
        messages: list[dict[str, str]] = []
        if self.system:
            messages.append({'role': 'system', 'content': self.system})
        if self.prompt:
            new_query = self.prompt.render(query=query, **self.kwargs)
            messages.append({'role': 'users', 'content': new_query})

        return await self.backend.arun(messages)

class CoT(SinglePasser):

    def __init__(self, backend: OpenAIBackend, prompt: Template = None, **kwargs):
        if not prompt:
            prompt = Environment(loader=FileSystemLoader(".")).get_template(
                "../prompt_weights/cot.j2"
            )
        super().__init__(backend, prompt, **kwargs)

    async def forward(self, query: str):
        answer=await super().forward(query)
        
